In [1]:
!pip install google-generativeai
!pip install python-dotenv

In [3]:
import google.generativeai as genai
import json
import re
import os
from dotenv import load_dotenv

load_dotenv()

# Set your API key
ci_api_key = os.getenv("CI_API_KEY")

# Configure the API key
genai.configure(api_key=ci_api_key)

# Function to detect emotional tone and assign one overall emotion
def generate_emotion(text):
    model = genai.GenerativeModel("gemini-2.0-flash")  # Define the model

    # Define input text
    input_text = f"""
    Analyze the emotional tone in the following financial text and categorize the emotions into the following categories:
    optimism, anxiety, sadness, surprise, neutral, anger/disgust. 
    Please output the emotion for each category, with a confidence score between 1 and 10 for each emotion. 
    If a category is not present in the text, do not include it in the response.

    The text is:
    {text}

    The output should be in the following format:
    {{
        "optimism": confidence_score (1-10),
        "anxiety": confidence_score (1-10),
        "sadness": confidence_score (1-10),
        "surprise": confidence_score (1-10),
        "neutral": confidence_score (1-10),
        "anger_disgust": confidence_score (1-10)
    }}
    """

    # Generate content from model
    response = model.generate_content(input_text)

    # Combine response text
    response_text = "".join([chunk.text for chunk in response])

    # Clean up the response
    cleaned_text = re.sub(r"```json|```", "", response_text).strip()

    # Convert response to JSON
    try:
        output_json = json.loads(cleaned_text)
    except json.JSONDecodeError:
        output_json = {"error": "Invalid JSON output from model", "raw_response": response_text}

    # Find the emotion with the highest confidence score
    if "error" not in output_json:
        # Find the emotion with the highest confidence score
        max_emotion = max(output_json, key=output_json.get)
        overall_emotion = {max_emotion: output_json[max_emotion]}
        return overall_emotion
    else:
        return output_json  # Return the error message if JSON was not valid

# Example usage
text = "The stock market showed significant growth today, and investors are optimistic about the future despite some minor concerns."

emotion_output = generate_emotion(text)
print(emotion_output)


{'optimism': 8}
